In [2]:
from nltk import Tree
import networkx as nx
import re
import spacy


nlp = spacy.load("zh_core_web_trf")
# nlp = spacy.load("zh_core_web_sm")
# nlp = spacy.load("en_core_web_sm")

In [32]:
doc = nlp("電影裡讓我直接想起來的還有——吳宇森導演的《辣手神探》的殮房的祕門啊")
entity1 = "吳宇森"
entity2 = "神探"
predicate = "導演"



print('sentence:'.format(doc))
# Load spacy's dependency tree into a networkx graph
edges = []
edges_and_dependencies = []
edges_list = []
dependencies_list = []
index_1 = []
index_2 = []
index_3 = []

for token in doc:
    for child in token.children:
        # append token to construct graph
        edges.append(('{0}'.format(token.lower_),
                      '{0}'.format(child.lower_)))
        edges_and_dependencies.append(('{0}'.format((token.lower_, token.dep_)),
                      '{0}'.format((child.lower_, child.dep_))))
    
    # append data for graph searchin
    edges_list.append(token)
    dependencies_list.append(token.dep_)
        
        
graph = nx.Graph(edges)
graph_test = nx.Graph(edges_and_dependencies)

# print(edges_list)
# print(dependencies_list)


for index, element in enumerate(edges_list):
    if str(element) == entity1:
        index_1.append(index)
    elif str(element) == entity2:
        index_2.append(index)
    elif str(element) == predicate:
        index_3.append(index)
        


if len(index_1) == len(index_2) == len(index_3)== 1:
    e_1 = "(\'" + str(edges_list[index_1[0]]) + "\', \'" + dependencies_list[index_1[0]] + "\')"
    e_2 = "(\'" + str(edges_list[index_2[0]]) + "\', \'" + dependencies_list[index_2[0]] + "\')"
    e_3 = "(\'" + str(edges_list[index_3[0]]) + "\', \'" + dependencies_list[index_3[0]] + "\')"
    shortest_path_list = nx.shortest_path(graph_test, source=e_1, target=e_3) + nx.shortest_path(graph_test, source=e_3, target=e_2)[1:]
else:
    for indexA in index_1:
        for indexB in index_2:
            for indexC in index_3:
                e_1 = "(\'" + str(edges_list[indexA]) + "\', \'" + dependencies_list[indexA] + "\')"
                e_2 = "(\'" + str(edges_list[indexB]) + "\', \'" + dependencies_list[indexB] + "\')"
                e_3 = "(\'" + str(edges_list[indexC]) + "\', \'" + dependencies_list[indexC] + "\')"
                shortest_path_list = nx.shortest_path(graph_test, source=e_1, target=e_2)[1:]
        
print(shortest_path_list)

sentence:
["('吳宇森', 'nsubj')", "('導演', 'acl')", "('祕門', 'dobj')", "('殮房', 'nmod:assmod')", "('神探', 'nmod:assmod')"]


In [21]:
doc = nlp("電影裡讓我直接想起來的還有——吳宇森導演的《辣手神探》的殮房的祕門啊")
entity1 = '吳宇森'
entity2 = '神探'


def find_element_nested_list(element, list_source, index_list, index="start"):
    # find element recursively
    for the_index, subList in enumerate(list_source):
        if type(subList) is not list:
            if list_source[the_index] == element:
                if index != "start": index_list.append(str(index))
                index_list.append(str(the_index))
                index_list.append("@")
        else:
            find_element_nested_list(element, subList, index_list, str(the_index))

    index_result = "-".join(index_list)
    index_result = re.split("|".join(["-@-", "-@"]), index_result)[:-1]

    return index_result


edges = []
edges_look_up = []
edges_and_dependencies_look_up = []
edges_and_dependencies = []



for token in doc:
    for child in token.children:
        edges_look_up.append([token.lower_,
                                      child.lower_])
        edges_and_dependencies_look_up.append([(token.lower_, token.dep_),
                                                       (child.lower_, child.dep_)])

        edges.append(('{0}'.format(token.lower_),
                              '{0}'.format(child.lower_)))
        edges_and_dependencies.append(('{0}'.format((token.lower_, token.dep_)),
                                               '{0}'.format((child.lower_, child.dep_))))

graph = nx.Graph(edges)
graph_test = nx.Graph(edges_and_dependencies)
        # Get the length and path

first_entity_index_list = find_element_nested_list(entity1, edges_look_up, [])
second_entity_index_list = find_element_nested_list(entity2, edges_look_up, [])

e_1 = edges_and_dependencies[int(first_entity_index_list[0].split("-")[0])][
            int(first_entity_index_list[0].split("-")[1])]
e_2 = edges_and_dependencies[int(second_entity_index_list[0].split("-")[0])][
            int(second_entity_index_list[0].split("-")[1])]

print(e_1, type(e_1))

shortest_path_list = nx.shortest_path(graph_test, source=e_1, target=e_2)

print(shortest_path_list)

('吳宇森', 'nsubj') <class 'str'>
["('吳宇森', 'nsubj')", "('導演', 'acl')", "('祕門', 'dobj')", "('殮房', 'nmod:assmod')", "('神探', 'nmod:assmod')"]


In [ ]:
# print(edges_look_up)


# first_entity_index_list = find_element_nested_list(entity1, edges_look_up, [])
# second_entity_index_list = find_element_nested_list(entity2, edges_look_up, [])
# e_1 = edges_and_dependencies[int(first_entity_index_list[0].split("-")[0])][int(first_entity_index_list[0].split("-")[1])]
# e_2 = edges_and_dependencies[int(second_entity_index_list[0].split("-")[0])][int(second_entity_index_list[0].split("-")[1])]

# print(e_1, e_2)